# Data mining untuk Thermal dan SO2 dari web http://mounts-project.com
---

## Import Modules python yang dibutuhkan
Adapun module yang digunakan antara lain:
1. Module `requests` digunakan untuk membuka web http://mounts-project.com
2. Module `json` digunakan untuk melakukan formatting data JSON
3. Module `re` digunakan untuk melakukan ekstraksi data JSON dari variable JavaScript menggunakan Regex
4. Module `pandas` digunakan untuk melakukan formatting data ke DataFrame dan converting ke Excel atau CSV
5. Module `os` digunakan untuk mendapatkan directory dan file
6. Module `JSON` digunakan untuk display data JSON

In [1]:
import requests
import json
import re
import pandas as pd
import os
from IPython.display import JSON
from datetime import datetime

## Inisiasi Variable yang Dibutuhkan
Inisiasi variable untuk web http://mounts-project.com

In [2]:
mounts_url = 'http://mounts-project.com/timeseries/'

Inisiasi variable untuk `output_directory` tempat hasil ekstrak data disimpan

In [3]:
output_directory = os.path.join(os.getcwd(), 'output')

output_directory

'E:\\Projects\\extract-mounts\\output'

In [4]:
if (not os.path.exists(output_directory)):
    os.mkdir(output_directory)

Variable `volcanoes` merupakan variable kode gunung api berdasarkan ID smithsonian number (https://volcano.si.edu/).  
Untuk mendapatkan data beberapa gunung api, masukkan kode gunung api dalam bentuk `list`

In [5]:
volcanoes = [
    {
        "name" : "Anak Krakatau",
        "code" : 262000,
    },
    {
        "name" : "Kerinci",
        "code" : 261170,
    },
    {
        "name" : "Karangetang",
        "code" : 267020,
    },
    {
        "name" : "Dukono",
        "code" : 268010,
    },
    {
        "name" : "Ili Lewotolok",
        "code" : 264230,
    },
    {
        "name" : "Ibu",
        "code" : 268030,
    },
    {
        "name" : "Semeru",
        "code" : 263300,
    },
    {
        "name" : "Raung",
        "code" : 263340,
    },
    {
        "name" : "Ijen",
        "code" : 263350,
    }
]

In [6]:
for index, volcano in enumerate(volcanoes):
    print(volcano['name'], volcano['code'])

Anak Krakatau 262000
Kerinci 261170
Karangetang 267020
Dukono 268010
Ili Lewotolok 264230
Ibu 268030
Semeru 263300
Raung 263340
Ijen 263350


Menginisiasi list kosong untuk menyimpan DataFrame hasil ekstraksi data. List ini akan berisi kumpulan data Thermal dan SO2 dari berbagai gunung api

In [7]:
dataframes = {}

---
## Inisiasi Fungsi-fungsi yang digunakan untuk extract, transformasi, dan export data
Fungsi yang digunakan untuk mengeskstrak variable JSON Thermal dan SO2 dari JavaScript web Mounts Project. Variabel ini tersimpan dengan nama `var_graph`

In [8]:
def get_json_from_javascript(response):
    var_graph = re.search(r"(?:^|\s|;)var\s+graph\s*=\s*([^']+})", response.text)
    string_graph = var_graph.group(1)
    json_graph = json.loads(string_graph)
    return json_graph

Hasil dari ekstraksi JSON pada fungsi `get_json_from_javascript` selanjutnya digunakan untuk extraksi nilai **SO2** dan **Thermal**

In [9]:
def get_so2_values(graph_json):
    so2_values = {
        'Date time': graph_json['data'][2]['x'],
        'Value': graph_json['data'][2]['y'],
    }
    so2_df = pd.DataFrame.from_dict(so2_values)
    so2_df['Type'] = 'SO2'
    return so2_df

In [10]:
def get_thermal_values(graph_json):
    thermal_values = {
        'Date time': graph_json['data'][0]['x'],
        'Value': graph_json['data'][0]['y'],
    }
    thermal_df = pd.DataFrame.from_dict(thermal_values)
    thermal_df['Type'] = 'Thermal'
    return thermal_df

Splitting kolom `Date Time` ke `Date` dan `Time`

In [11]:
def convert_to_date(date_time):
    return date_time.strftime("%Y-%m-%d")

In [12]:
def convert_to_time(date_time):
    return date_time.strftime("%H:%M:%S")

Fungsi ini digunakan untuk meng-export data hasil ekstraksi ke dalam format Excel

In [13]:
def export_to_excel(filtered_df, volcano_code, volcano_name):
    filename = '{} - {}.xlsx'.format(volcano_name, volcano_code)
    path = os.path.join(output_directory, filename)
    filtered_df.to_excel(path, sheet_name='Join Data')
    return path

---
## Kode Utama
Kode utama ekstraksi data

In [20]:
df_excel = pd.DataFrame()

for index, volcano in enumerate(volcanoes):
    volcano_code = volcano['code']
    volcano_name = volcano['name']
    
    # http://mounts-project.com/timeseries/262000
    url = mounts_url+str(volcano_code) 
    
    # Buka http://mounts-project.com/timeseries/262000
    response = requests.get(url)
    
    # Get data JSON
    graph_json = get_json_from_javascript(response)
    
    so2 = get_so2_values(graph_json)
    thermal = get_thermal_values(graph_json)
    
    df = pd.concat([
        so2,
        thermal
    ])
    
    df['Date time'] = pd.to_datetime(df['Date time'])
    df['Date'] = df['Date time'].apply(convert_to_date)
    df['Time'] = df['Date time'].apply(convert_to_time)
    df['Code'] = volcano_code
    df.set_index('Date time', inplace=True)
    
    filtered_df = df[df['Value'] > 0.1]
    dataframes[volcano_name] = filtered_df
    
    df_excel = pd.concat([
        df_excel, pd.DataFrame([{
            "code" : volcano_code,
            "volcano_name" : volcano_name,
            "filename" : export_to_excel(filtered_df, volcano_code, volcano_name),
            "updated_at" : filtered_df.index.max()
    }])], ignore_index=True)
    
    print("Data untuk gunung api {} berhasil diekstrak, dan disimpan dalam variable dataframes['{}']".format(volcano_name,volcano_name))

Data untuk gunung api Anak Krakatau berhasil diekstrak, dan disimpan dalam variable dataframes['Anak Krakatau']
Data untuk gunung api Kerinci berhasil diekstrak, dan disimpan dalam variable dataframes['Kerinci']
Data untuk gunung api Karangetang berhasil diekstrak, dan disimpan dalam variable dataframes['Karangetang']
Data untuk gunung api Dukono berhasil diekstrak, dan disimpan dalam variable dataframes['Dukono']
Data untuk gunung api Ili Lewotolok berhasil diekstrak, dan disimpan dalam variable dataframes['Ili Lewotolok']
Data untuk gunung api Ibu berhasil diekstrak, dan disimpan dalam variable dataframes['Ibu']
Data untuk gunung api Semeru berhasil diekstrak, dan disimpan dalam variable dataframes['Semeru']
Data untuk gunung api Raung berhasil diekstrak, dan disimpan dalam variable dataframes['Raung']
Data untuk gunung api Ijen berhasil diekstrak, dan disimpan dalam variable dataframes['Ijen']


In [21]:
df_excel

,code,volcano_name,filename,updated_at
0,262000,Anak Krakatau,E:\Projects\extract-mounts\output\Anak Krakata...,2023-01-22 06:08:00
1,261170,Kerinci,E:\Projects\extract-mounts\output\Kerinci - 26...,2023-01-18 07:23:00
2,267020,Karangetang,E:\Projects\extract-mounts\output\Karangetang ...,2023-01-19 05:23:00
3,268010,Dukono,E:\Projects\extract-mounts\output\Dukono - 268...,2023-01-22 04:28:00
4,264230,Ili Lewotolok,E:\Projects\extract-mounts\output\Ili Lewotolo...,2023-01-18 02:04:39
5,268030,Ibu,E:\Projects\extract-mounts\output\Ibu - 268030...,2023-01-22 04:28:00
6,263300,Semeru,E:\Projects\extract-mounts\output\Semeru - 263...,2023-01-22 06:08:00
7,263340,Raung,E:\Projects\extract-mounts\output\Raung - 2633...,2023-01-23 05:48:00
8,263350,Ijen,E:\Projects\extract-mounts\output\Ijen - 26335...,2023-01-16 06:18:59


In [22]:
df_excel.to_excel('output.xlsx', index=False)